**Some first imports**

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import requests
import json
import torch
import os
from tqdm import tqdm


In [3]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

In [4]:
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

File ‘train-v2.0.json’ already there; not retrieving.

File ‘dev-v2.0.json’ already there; not retrieving.



In [5]:
# Load the training dataset and take a look at it
with open('train-v2.0.json', 'rb') as f:
  squad = json.load(f)

In [5]:
# Each 'data' dict has two keys (title and paragraphs)
squad['data'][0].keys()

dict_keys(['title', 'paragraphs'])

In [ ]:
# Find the group about Greece
gr = -1
for idx, group in enumerate(squad['data']):
  print(group['title'])
  if group['title'] == 'Greece':
    gr = idx
    print(gr)
    break

In [ ]:
# let's check on Greece which is 186th (0-based indexing)
# we can see that we have a context and many questions and answers following
squad['data'][186]

In [ ]:
# and this is the context given for NYC
squad['data'][186]['paragraphs'][0]['context']

In [ ]:
#### this context has unaswerable questions
squad['data'][441]['paragraphs'][28]['context']

In [ ]:
squad['data'][441]['paragraphs'][28]['qas']

In [6]:
def read_data(path): ## Just work with Answerable data Because we wanna to generate UNANSWERABLE pairs based on the ANSWERABLE ones
  # load the json file
  with open(path, 'rb') as f:
    squad = json.load(f)

  contexts = []
  questions = []
  answers = [] ## Multi-Labels based on SQuAD2_CR dataset
  answers_spans = [] ## For ANSWERABLE Questions: it has answwer span and its start index within the context
  qids = []

  for group in squad['data']:
    for passage in group['paragraphs']:
      context = passage['context']
      for qa in passage['qas']:
        if qa["is_impossible"] == False:
          qid = qa["id"]
          qids.append(qid)

          question = qa['question']
          questions.append(question)

          contexts.append(context)

          answer_span = (qa["answers"][0]["text"] , qa["answers"][0]["answer_start"])
          answers_spans.append(answer_span)

          answers.append("Ans")

  data_all = {'qid': qids,
          'context': contexts,
          'question': questions,
          'labels' : answers,
            'answer span' : answers_spans}

  # Convert the dictionary into DataFrame
  df_all = pd.DataFrame(data_all)
  return df_all




Put the contexts, questions and answers for training and validation into the appropriate lists.

In [7]:
from collections import Counter
print("TRAIN : ")
train_df_all = read_data('train-v2.0.json')
display(train_df_all)
# counter1 = Counter(train_answers)
# print('train_answers : ' , counter1)
print("DEV : ")
dev_df_all = read_data('dev-v2.0.json')
# counter2 = Counter(dev_answers)
# print('dev_answers : ' , counter2)
display(dev_df_all)


TRAIN : 


,qid,context,question,labels,answer span
0,56be85543aeaaa14008c9063,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,Ans,"(in the late 1990s, 269)"
1,56be85543aeaaa14008c9065,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,Ans,"(singing and dancing, 207)"
2,56be85543aeaaa14008c9066,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,Ans,"(2003, 526)"
3,56bf6b0f3aeaaa14008c9601,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,Ans,"(Houston, Texas, 166)"
4,56bf6b0f3aeaaa14008c9602,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,Ans,"(late 1990s, 276)"
...,...,...,...,...,...
86816,5735d259012e2f140011a09d,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,Ans,"(Oregon, 229)"
86817,5735d259012e2f140011a09e,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,Ans,"(Rangoon, 414)"
86818,5735d259012e2f140011a09f,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,Ans,"(Minsk, 476)"
86819,5735d259012e2f140011a0a0,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,Ans,"(1975, 199)"


DEV : 


,qid,context,question,labels,answer span
0,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,Ans,"(France, 159)"
1,56ddde6b9a695914005b9629,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,Ans,"(10th and 11th centuries, 94)"
2,56ddde6b9a695914005b962a,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,Ans,"(Denmark, Iceland and Norway, 256)"
3,56ddde6b9a695914005b962b,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,Ans,"(Rollo, 308)"
4,56ddde6b9a695914005b962c,The Normans (Norman: Nourmands; French: Norman...,What century did the Normans first gain their ...,Ans,"(10th century, 671)"
...,...,...,...,...,...
5923,5737aafd1c456719005744fb,"The pound-force has a metric counterpart, less...",What is the metric term less used than the New...,Ans,"(kilogram-force, 82)"
5924,5737aafd1c456719005744fc,"The pound-force has a metric counterpart, less...",What is the kilogram-force sometimes reffered ...,Ans,"(kilopond, 114)"
5925,5737aafd1c456719005744fd,"The pound-force has a metric counterpart, less...",What is a very seldom used unit of mass in the...,Ans,"(slug, 274)"
5926,5737aafd1c456719005744fe,"The pound-force has a metric counterpart, less...",What seldom used term of a unit of force equal...,Ans,"(kip, 712)"


In [ ]:
train_df_all.to_csv('train_df_all.csv')
train_df_all

,qid,Context,question,answers
0,56be85543aeaaa14008c9063,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,Ans
1,56be85543aeaaa14008c9065,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,Ans
2,56be85543aeaaa14008c9066,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,Ans
3,56bf6b0f3aeaaa14008c9601,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,Ans
4,56bf6b0f3aeaaa14008c9602,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,Ans
...,...,...,...,...
130314,5a7e070b70df9f001a875439,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,A
130315,5a7e070b70df9f001a87543a,"The term ""matter"" is used throughout physics i...",Who coined the term partonic matter?,E
130316,5a7e070b70df9f001a87543b,"The term ""matter"" is used throughout physics i...",What is another name for anti-matter?,E
130317,5a7e070b70df9f001a87543c,"The term ""matter"" is used throughout physics i...",Matter usually does not need to be used in con...,N


In [ ]:
dev_df_all.to_csv('dev_df_all.csv')
dev_df_all

,qid,Context,question,answers
0,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,Ans
1,56ddde6b9a695914005b9629,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,Ans
2,56ddde6b9a695914005b962a,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,Ans
3,56ddde6b9a695914005b962b,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,Ans
4,56ddde6b9a695914005b962c,The Normans (Norman: Nourmands; French: Norman...,What century did the Normans first gain their ...,Ans
...,...,...,...,...
11868,5737aafd1c456719005744ff,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,Ans
11869,5ad28ad0d7d075001a4299cc,"The pound-force has a metric counterpart, less...",What does not have a metric counterpart?,N
11870,5ad28ad0d7d075001a4299cd,"The pound-force has a metric counterpart, less...",What is the force exerted by standard gravity ...,E
11871,5ad28ad0d7d075001a4299ce,"The pound-force has a metric counterpart, less...",What force leads to a commonly used unit of mass?,E


In [37]:
train_df_split = train_df_all
(train_df_split)


,qid,context,question,labels,answer span
0,56be85543aeaaa14008c9063,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,Ans,"(in the late 1990s, 269)"
1,56be85543aeaaa14008c9065,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,Ans,"(singing and dancing, 207)"
2,56be85543aeaaa14008c9066,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,Ans,"(2003, 526)"
3,56bf6b0f3aeaaa14008c9601,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,Ans,"(Houston, Texas, 166)"
4,56bf6b0f3aeaaa14008c9602,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,Ans,"(late 1990s, 276)"
...,...,...,...,...,...
86816,5735d259012e2f140011a09d,"Kathmandu Metropolitan City (KMC), in order to...",In what US state did Kathmandu first establish...,Ans,"(Oregon, 229)"
86817,5735d259012e2f140011a09e,"Kathmandu Metropolitan City (KMC), in order to...",What was Yangon previously known as?,Ans,"(Rangoon, 414)"
86818,5735d259012e2f140011a09f,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,Ans,"(Minsk, 476)"
86819,5735d259012e2f140011a0a0,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,Ans,"(1975, 199)"


## **Antonym**

In [24]:
import nltk
from nltk.corpus import wordnet
import random

def get_antonym(word):
    antonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            if lemma.antonyms():
                antonyms.add(lemma.antonyms()[0].name())
    return list(antonyms)

def generate_modified_question(original_question):
    original_words = original_question.split()
    modified_questions = []
    for i, word in enumerate(original_words):
        antonyms = get_antonym(word)
        if antonyms:
            for antonym in antonyms:
                modified_question = original_words.copy()
                modified_question[i] = antonym
                modified_questions.append(' '.join(modified_question))
    return modified_questions

# Example usage
original_question = "Time magazine named her one of the most 100 what people of the century?"

modified_questions = generate_modified_question(original_question)
print("Original Question:", original_question)
print("Modified Questions:")
for modified_question in modified_questions:
    print(modified_question)


Original Question: Time magazine named her one of the most 100 what people of the century?
Modified Questions:
Time magazine named her one of the fewest 100 what people of the century?
Time magazine named her one of the least 100 what people of the century?


In [38]:
import spacy
import re
from spacy import displacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import textdistance as td
import pandas as pd
import nltk
from nltk.corpus import wordnet
import random
nltk.download('wordnet')

def get_antonym(word):
    antonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            if lemma.antonyms():
                antonyms.add(lemma.antonyms()[0].name())
    return list(antonyms)


modified_Antonym_df = pd.DataFrame(columns=["qid", "context", "original question","original answer span" , "original label" ,"modified question"])

for index, row in train_df_split.iterrows():
    if index % 1000 == 0:
        print(index)
    id = row["qid"]
    con = row["context"]
    q = row["question"]
    lbl = row["labels"]
    ans_span = row["answer span"]
    
    q_words = q.split()
    
    
    for i in range(len(q_words)):
        antonyms = get_antonym(q_words[i])

        if antonyms:
            for antonym in antonyms:
                modified_question = q_words.copy()
                modified_question[i] = antonym
                modified_question= ' '.join(modified_question)

                modified_Antonym_df.loc[len(modified_Antonym_df.index)] = (id,con,q,ans_span,lbl,modified_question)
    

display(modified_Antonym_df)

[nltk_data] Downloading package wordnet to /home/hmoradis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000


,qid,context,original question,original answer span,original label,modified question
0,56be85543aeaaa14008c9063,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"(in the late 1990s, 269)",Ans,When unmake Beyonce start becoming popular?
1,56be85543aeaaa14008c9063,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"(in the late 1990s, 269)",Ans,When did Beyonce finish becoming popular?
2,56be85543aeaaa14008c9063,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"(in the late 1990s, 269)",Ans,When did Beyonce middle becoming popular?
3,56be85543aeaaa14008c9063,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"(in the late 1990s, 269)",Ans,When did Beyonce stop becoming popular?
4,56be85543aeaaa14008c9063,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"(in the late 1990s, 269)",Ans,When did Beyonce end becoming popular?
...,...,...,...,...,...,...
324809,5735d259012e2f140011a09f,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,"(Minsk, 476)",Ans,With what Belorussian city does Kathmandu refu...
324810,5735d259012e2f140011a09f,"Kathmandu Metropolitan City (KMC), in order to...",With what Belorussian city does Kathmandu have...,"(Minsk, 476)",Ans,With what Belorussian city does Kathmandu lack...
324811,5735d259012e2f140011a0a0,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,"(1975, 199)",Ans,In what year unmake Kathmandu create its initi...
324812,5735d259012e2f140011a0a0,"Kathmandu Metropolitan City (KMC), in order to...",In what year did Kathmandu create its initial ...,"(1975, 199)",Ans,In what year did Kathmandu create its initial ...


In [39]:
modified_Antonym_df.to_csv('modified_Antonym_allData_df.csv')

In [8]:
import spacy
import re
from spacy import displacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import textdistance as td
import pandas as pd
import nltk
from nltk.corpus import wordnet
import random
nltk.download('wordnet')

def get_antonym(word):
    antonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            if lemma.antonyms():
                antonyms.add(lemma.antonyms()[0].name())
    return list(antonyms)




dev_modified_Antonym_df = pd.DataFrame(columns=["qid", "context", "original question","original answer span" , "original label" ,"modified question"])

for index, row in dev_df_all.iterrows():
    if index % 1000 == 0:
        print(index)
    id = row["qid"]
    con = row["context"]
    q = row["question"]
    lbl = row["labels"]
    ans_span = row["answer span"]
    
    q_words = q.split()
    
    
    for i in range(len(q_words)):
        antonyms = get_antonym(q_words[i])

        if antonyms:
            for antonym in antonyms:
                modified_question = q_words.copy()
                modified_question[i] = antonym
                modified_question= ' '.join(modified_question)

                dev_modified_Antonym_df.loc[len(dev_modified_Antonym_df.index)] = (id,con,q,ans_span,lbl,modified_question)
    

display(dev_modified_Antonym_df)

[nltk_data] Downloading package wordnet to /home/hmoradis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0
1000
2000
3000
4000
5000


,qid,context,original question,original answer span,original label,modified question
0,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,"(France, 159)",Ans,In what urban_area is Normandy located?
1,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,"(France, 159)",Ans,In what country differ Normandy located?
2,56ddde6b9a695914005b9629,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,"(10th and 11th centuries, 94)",Ans,When differ the Normans in Normandy?
3,56ddde6b9a695914005b962a,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,"(Denmark, Iceland and Norway, 256)",Ans,From which urban_area did the Norse originate?
4,56ddde6b9a695914005b962a,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,"(Denmark, Iceland and Norway, 256)",Ans,From which countries unmake the Norse originate?
...,...,...,...,...,...,...
23489,5737aafd1c456719005744ff,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,"(sthène, 665)",Ans,What is the seldom used pull unit equal to one...
23490,5737aafd1c456719005744ff,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,"(sthène, 665)",Ans,What is the seldom used push unit equal to one...
23491,5737aafd1c456719005744ff,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,"(sthène, 665)",Ans,What is the seldom used force unit inadequate ...
23492,5737aafd1c456719005744ff,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,"(sthène, 665)",Ans,What is the seldom used force unit differ to o...


In [9]:
dev_modified_Antonym_df.to_csv('dev_modified_Antonym_allData_df.csv')